# Environment Setup
This section sets up environment variables for API endpoints and keys.

In [3]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ['LLM_ENDPOINT'] = os.getenv('LLM_ENDPOINT')

os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')



# Load and Parse Web Page
This section loads the web page and parses its content using BeautifulSoup and LangChain's WebBaseLoader.

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Split Documents into Chunks
This section splits the loaded documents into manageable text chunks for embedding and retrieval.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

# Create Vector Store and Generate Embeddings
This section creates a Chroma vector store and generates embeddings for the text chunks using Ollama.

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OllamaEmbeddings(model="qwen3:14b")
)

# Cleanup Vector Store
This section deletes the Chroma vector store collection to free up resources.

In [ ]:
#vectorstore.delete_collection()

# Retrieve and Print Document
This section retrieves a document relevant to the query and prints its content.

In [8]:
retriever = vectorstore.as_retriever()

docs = retriever.invoke("What is Task Decomposition?")

print(docs[0].page_content)

Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-specific PDDL and a suitable planner which is common in certain robotic setups but not in many other domains.
Self-Reflection#


# Retrieve Prompt from LangChain Hub
This section pulls a prompt from the LangChain hub and prints it.

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


# Initialize Ollama LLM
This section initializes the Ollama language model (LLM) with the specified model and temperature.

In [9]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen3:14b", temperature=0)

# Build RAG Pipeline
This section defines the output parser, document formatting function, and constructs the Retrieval-Augmented Generation (RAG) chain.

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser()
)

# Run RAG Chain and Display Response
This section invokes the RAG chain with a sample question and prints the generated response.

In [11]:
response = rag_chain.invoke("What is Task Decomposition?")
print(response)

<think>
Okay, the user is asking about Task Decomposition. Let me check the context provided.

The first part of the context mentions that task decomposition can be done by LLM with simple prompting, using task-specific instructions, or with human inputs. Then there's another approach called LLM+P which uses an external planner and PDDL. The answer should cover these methods briefly.

I need to make sure I don't include the other sections about risks, generative agents, or APIs since they aren't related to task decomposition. The answer should be concise, three sentences max. Let me structure it: define task decomposition, mention the methods (LLM prompting, task-specific instructions, human input), and then the LLM+P approach with PDDL. That should cover it without going over the limit.
</think>

Task decomposition is the process of breaking down a complex task into smaller, manageable subtasks. It can be achieved through LLM prompting, task-specific instructions, or human input. An a